In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import statsmodels.api as sm
import seaborn as sns
import mne
import openpyxl as op
import matplotlib.cm as cm

# Figure 2

## 4분면으로 그리기

### 0327

In [ ]:
# 사분면 플롯

import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import matplotlib.patches as patches
 
# 파일 경로 및 "pharmacoEEG" 시트 읽기 
file_path = r"D:\주성\documents\2025\Mg comma\차트리뷰\clinical features11.xlsx" 
pharmacoEEG_df = pd.read_excel(file_path, sheet_name="pharmacoEEG") 
clinical_df = pd.read_excel(file_path, sheet_name="clinical_data") 
clinical_df = clinical_df.rename(columns={"Patient No.": "ID"}) 
target_response = "response_10_and" # 타깃하는 response 기준 선택 
 
# band 컬럼의 고유값별로 데이터 분리 
bands = pharmacoEEG_df['band'].unique() 
 
# 각 band별로 산점도 그리기 
for band in bands: 
    # 현재 band에 해당하는 데이터 선택 
    df_band = pharmacoEEG_df[pharmacoEEG_df['band'] == band].copy() 
     
    # post_change와 day1_change에 100을 곱하여 % 단위 변환 
    df_band['post_change'] *= 100 
    df_band['day1_change'] *= 100 
     
    # response 정보를 ID 기준으로 매핑 
    df_band = df_band.merge(clinical_df[['ID', target_response]], on='ID', how='left') 
     
    # day1_change가 결측치가 아닌 데이터 (정상 값) 
    valid_df = df_band[df_band['day1_change'].notna()] 
    # day1_change가 결측치인 데이터: y값은 0으로 대체하고 투명도를 조절하여 표시 
    missing_df = df_band[df_band['day1_change'].isna()].copy() 
    missing_df['day1_change'] = 0  # 결측치 0으로 대체 
     
    # 그림 크기를 더 크게 설정하여 축 라벨이 겹치지 않도록 함
    fig, ax = plt.subplots(figsize=(10, 8)) 
    
    # 응답 영역을 노란색으로 표시 (-10% 이하 영역)
    # 1. 왼쪽 아래 사각형 (-100,-100)에서 (-10,-10)까지
    response_area = patches.Rectangle((-100, -100), 90, 90, alpha=0.2, color='yellow')
    ax.add_patch(response_area)
     
    # 범례를 위한 빈 플롯 생성
    responder_plot = ax.scatter([], [], color='blue', alpha=0.7, label='Responder')
    non_responder_plot = ax.scatter([], [], color='red', alpha=0.7, label='Non-responder')
    day1_missing_plot = ax.scatter([], [], color='gray', alpha=0.5, label='Day1-missing')
    
    # 유효한 day1_change 값을 가진 점들을 response 값에 따라 색상 구분하여 표시 
    for _, row in valid_df.iterrows(): 
        color = 'blue' if row[target_response] else 'red' 
        ax.scatter(row['post_change'], row['day1_change'], color=color, alpha=0.7) 
        ax.text(row['post_change'], row['day1_change'], str(row['ID']), fontsize=9, ha='right', va='bottom', color=color) 
     
    # NA인 day1_change 값은 회색으로 표시하고, 투명도를 50%로 설정 
    for _, row in missing_df.iterrows(): 
        color = 'gray'  # 회색으로 변경
        ax.scatter(row['post_change'], row['day1_change'], color=color, alpha=0.5) 
        ax.text(row['post_change'], row['day1_change'], str(row['ID']), fontsize=9, ha='left', va='top', color=color, alpha=0.5) 
     
    # x축과 y축을 원점(0,0) 기준으로 설정 
    ax.spines['left'].set_position('zero') 
    ax.spines['bottom'].set_position('zero') 
    ax.spines['right'].set_color('none') 
    ax.spines['top'].set_color('none') 
     
    # 양쪽 축에 눈금 표시 
    ax.xaxis.set_ticks_position('bottom') 
    ax.yaxis.set_ticks_position('left') 
     
    # x축과 y축의 스케일을 -100에서 200으로 통일하고 눈금 간격 동일하게 설정
    ax.set_xlim(-100, 200) 
    ax.set_ylim(-100, 200)
    
    # 축 눈금 간격 설정 - 두 축 모두 동일한 간격으로
    major_ticks = np.arange(-100, 201, 50)  # -100부터 200까지 50 간격
    ax.set_xticks(major_ticks)
    ax.set_yticks(major_ticks)
    
    # 종횡비 1:1로 설정하여 x와 y 스케일이 시각적으로 동일하게 보이도록
    ax.set_aspect('equal')
     
    # x = -10, y = -10 위치에 회색 점선 추가 
    ax.axvline(x=-10, linestyle='--', color='grey') 
    ax.axhline(y=-10, linestyle='--', color='grey') 
     
    # 범례 위치 수정 - 그래프 외부 우측 상단에 위치
    ax.legend(loc='upper right', bbox_to_anchor=(1.25, 1.0))
    
    ax.set_title(f'Band: {band}', pad=20) 
    
    # x축 라벨 위치 조정 - 그래프 아래쪽에 충분한 간격을 두고 배치
    ax.xaxis.set_label_coords(0.5, -0.12)  # x축 라벨을 아래로 이동
    ax.set_xlabel('Relative Change in Power at Post-infusion (%)')
    
    # y축 라벨 위치 조정 - 왼쪽으로 충분히 이동
    ax.yaxis.set_label_coords(-0.12, 0.5)  # y축 라벨을 왼쪽으로 이동
    ax.set_ylabel('Relative Change in Power at 24h-infusion (%)')
    
    ax.grid(True) 
    
    # 여백을 더 넓게 설정하여 라벨이 잘리지 않도록 함
    plt.subplots_adjust(left=0.2, bottom=0.15, right=0.85, top=0.9)
    plt.show()

## 스파게티 플롯

### 0314

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm  # For colormap

# Load data
file_path = r"D:\주성\documents\2025\Mg comma\차트리뷰\clinical features9.xlsx"
sheet_name_power = 'pharmacoEEG'
sheet_name_feature = 'clinical_data'

df = pd.read_excel(file_path, sheet_name=sheet_name_power)
response = pd.read_excel(file_path, sheet_name=sheet_name_feature)[['Patient No.', 'response_10']]
response.rename(columns={"Patient No.": "ID", "response_10": 'response'}, inplace=True)
df = df.merge(response, on='ID', how='left')

# Extract unique bands
bands = df["band"].unique()

# Set colormap (True = Blue scale, False = Red scale)
cmap_true = cm.get_cmap("Blues")  # Response=True group
cmap_false = cm.get_cmap("Reds")  # Response=False group

# Assign rankings separately for each band
df["rank_true"] = np.nan
df["rank_false"] = np.nan

for band in bands:
    band_data = df[df["band"] == band]

    # Rank within each band separately
    true_group = band_data[band_data["response"] == True]
    if not true_group.empty:
        df.loc[true_group.index, "rank_true"] = true_group["post_change"].abs().rank(method="min", pct=True)

    false_group = band_data[band_data["response"] == False]
    if not false_group.empty:
        df.loc[false_group.index, "rank_false"] = false_group["post_change"].abs().rank(method="min", pct=True)

# Plot change rate trends for each band
for band in bands:
    fig, ax = plt.subplots(figsize=(12, 6))  # Define Axes explicitly
    
    # Filter data for the current band
    band_data = df[df["band"] == band]

    # Plot individual patient change rates
    for _, row in band_data.iterrows():
        patient_id = row["ID"]
        pre_change = 0  # Pre-timepoint is always 0
        post_change = row["post_change"]
        day1_change = row["day1_change"]
        response_status = row["response"]

        # Assign colors based on rank within each band
        if response_status:
            norm_value = row["rank_true"] if not pd.isna(row["rank_true"]) else 0.5  # Default to mid-value
            color = cmap_true(norm_value)
        else:
            norm_value = row["rank_false"] if not pd.isna(row["rank_false"]) else 0.5  # Default to mid-value
            color = cmap_false(norm_value)

        # Adjust line thickness based on rank
        line_width = 1 + (norm_value * 4)  # Minimum width 1, max width 5
        marker_size = 6  # Keep marker size constant

        # Plot the change rate
        ax.plot(["pre", "post", "day1"], [pre_change, post_change, day1_change],
                 marker='o', linestyle='-', color=color, linewidth=line_width, markersize=marker_size, alpha=0.8)

    # Set plot title and labels
    ax.set_title(f"Change Rate Trend - {band} Band (Rank-based Coloring)", fontsize=14)
    ax.set_xlabel("Time", fontsize=12)
    ax.set_ylabel("Change Rate", fontsize=12)
    ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    ax.grid(True)

    # ✅ Set colorbar with actual percentile rank
    sm_true = cm.ScalarMappable(cmap=cmap_true, norm=plt.Normalize(vmin=0, vmax=1))
    sm_false = cm.ScalarMappable(cmap=cmap_false, norm=plt.Normalize(vmin=0, vmax=1))

    # ✅ Adjust colorbar labels for correct rank representation
    cbar_true = plt.colorbar(sm_true, ax=ax, fraction=0.03, pad=0.02)
    cbar_true.set_label("Percentile Rank (Response: True)")

    cbar_false = plt.colorbar(sm_false, ax=ax, fraction=0.03, pad=0.06)
    cbar_false.set_label("Percentile Rank (Response: False)")

    plt.show()


### 0310

In [ ]:
file_path = r"D:\주성\documents\2025\Mg comma\차트리뷰\clinical features9.xlsx"
sheet_name = 'pharmacoEEG'

df = pd.read_excel(file_path, sheet_name = sheet_name)

# 밴드별로 데이터 분리하기
bands = df['band'].unique()  # 모든 고유한 밴드 값 추출

# 그래프 스타일 설정
plt.style.use('ggplot')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.size'] = 10

# 밴드 이름 첫 글자 대문자로 변경하는 함수
def capitalize_band(band_name):
    return band_name.capitalize()

# 큰 도표 생성 (3x2 그리드)
fig, axes = plt.subplots(3, 2, figsize=(18, 15))
axes = axes.flatten()  # 2D 배열을 1D로 변환하여 접근 용이하게 함

# 각 밴드별 그래프 생성 (첫 5개 서브플롯)
for i, band in enumerate(bands[:5]):  # 5개 밴드까지만 처리
    # 해당 밴드의 데이터만 필터링
    band_data = df[df['band'] == band].copy()
    
    # 각 환자 ID별 데이터 시각화
    for patient_id in band_data['ID'].unique():
        patient_data = band_data[band_data['ID'] == patient_id]
        
        # 시간 포인트를 x축으로 사용하기 위한 데이터 준비
        time_points = ['pre', 'post', 'day1']
        changes = [
            0,  # pre는 기준점이므로 변화율은 0
            patient_data['post_change'].values[0] if not patient_data['post_change'].isna().all() else np.nan,
            patient_data['day1_change'].values[0] if not patient_data['day1_change'].isna().all() else np.nan
        ]
        
        # 결측치가 있는 부분은 점선으로 표시
        valid_indices = ~np.isnan(changes)
        
        # 선 그래프로 표시 (결측치가 없는 부분)
        axes[i].plot(
            np.array(time_points)[valid_indices], 
            np.array(changes)[valid_indices], 
            marker='o', 
            label=f'ID: {patient_id}'
        )
        
        # 결측치가 있는 경우 점선으로 연결
        if not all(valid_indices):
            for j in range(len(time_points)-1):
                if valid_indices[j] and valid_indices[j+1]:
                    continue  # 두 지점 모두 유효하면 실선으로 이미 그려짐
                if valid_indices[j] or valid_indices[j+1]:  # 적어도 하나는 유효
                    axes[i].plot(
                        [time_points[j], time_points[j+1]], 
                        [changes[j], changes[j+1]], 
                        linestyle='--', 
                        alpha=0.5, 
                        color='gray'
                    )
    
    # 그래프 꾸미기
    axes[i].axhline(y=0, color='black', linestyle='-', alpha=0.3)  # 0 기준선 추가
    axes[i].set_title(f'{capitalize_band(band)} Band Power Change Rate by Patient ID')
    axes[i].set_xlabel('Time Point')
    axes[i].set_ylabel('Relative Change in Power (%)')
    axes[i].set_xticks(range(len(time_points)))
    axes[i].set_xticklabels(time_points)
    
    # 환자 ID가 많은 경우 범례를 그래프 밖에 표시하지 않고 제외
    if len(band_data['ID'].unique()) <= 10:
        axes[i].legend(fontsize='small')

# 마지막 서브플롯(6번째)에 박스플롯 두 개 그리기
# Post 변화율에 대한 박스플롯
box_data = pd.melt(df, id_vars=['band'], value_vars=['post_change', 'day1_change'], 
                   var_name='time_point', value_name='change_rate')
box_data['time_point'] = box_data['time_point'].map({'post_change': 'Post', 'day1_change': 'Day 1'})

# 밴드 이름 첫 글자 대문자로 변경
box_data['band'] = box_data['band'].apply(capitalize_band)

sns.boxplot(x='band', y='change_rate', hue='time_point', data=box_data, ax=axes[5])
axes[5].set_title('Change Rate by Band and Time Point')
axes[5].set_xlabel('Band')
axes[5].set_ylabel('Relative Change in Power')
axes[5].axhline(y=0, color='red', linestyle='--')
axes[5].legend(title='Time Point')

# 전체 레이아웃 조정
plt.tight_layout()
plt.savefig('all_bands_change_rate.png', dpi=300)
plt.show()

## 오름차순 막대그래프

In [ ]:
df = pd.read_csv(r"E:\주성\documents\2025\Mg comma\data2.csv")

# CSV 파일 읽기
data = df

# 밴드별로 데이터 나누기
bands = ['delta', 'theta', 'alpha', 'beta', 'gamma']

plt.figure(figsize=(15, 10))
for idx, band in enumerate(bands, 1):
    plt.subplot(2, 3, idx)
    
    # 해당 밴드의 데이터만 추출
    band_data = data[data['band'] == band].copy()
    
    # 변화율 계산 ((post - pre) / pre)
    change = (band_data['mean_post_infusion'] - band_data['mean_pre_infusion']) / band_data['mean_pre_infusion']
    
    # 오름차순 정렬
    sorted_change = np.sort(change)
    
    # 바 차트 그리기
    plt.bar(range(len(sorted_change)), sorted_change)
    plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    plt.title(f'{band.capitalize()} Band')
    plt.ylim(-1, 2)  # y축 범위 수정

plt.suptitle('Power Change by Frequency Band', fontsize=14, y=1.02)    
plt.tight_layout()
plt.show()

# Table 3

## 기술적 통계값 구하고, 엑셀로 내보내는 코드

In [ ]:
import pandas as pd
import numpy as np

# 변수 설정 부분

# clinical feature 파일의 경로 설정
input_file_path = r"D:\주성\documents\2025\Mg comma\차트리뷰\clinical features12.xlsx"

# 결과를 저장할 엑셀 파일의 경로 설정
output_file_path = r"C:\Users\Brain_Science\Documents\박주성\Mg comma\표그림\new.xlsx"

# 분석할 response 컬럼 설정
target_response = 'response_10_and'

# clinical feature 불러오기
feature = pd.read_excel(input_file_path, sheet_name='clinical_data')
# 결측치 제거 - 수정된 부분: 결과를 feature에 다시 할당
feature = feature.dropna(subset=[target_response])

# 파생변수 생성. 파생변수 추가 시 여기에다가 추가
feature['STESS_5'] = feature['STESS'] >= 5
feature['STESS_3'] = feature['STESS'] >= 3
feature['mRS_premorbid_4'] = feature['mRS_premorbid'] >= 4
feature['mRS_premorbid_3'] = feature['mRS_premorbid'] >= 3
feature['Age_65'] = feature['Age'] >= 65
feature['mRS_discharge_4'] = feature['mRS_discharge'] >= 4
feature['mRS_discharge_3'] = feature['mRS_discharge'] >= 3
feature['mRS_3mo_4'] = feature['mRS_3mo'] >= 4
feature['mRS_3mo_3'] = feature['mRS_3mo'] >= 3
feature['Seizure_terminated'] = ~feature['Seizure_not_terminated']

# 범주가 2개인 변수 설정
bi_var = ['SRSE', 'CIVAD', 'Sex', 'ImmunoTx',
    'NORSE', 'Mg_protocol', 'STESS_5', 'STESS_3', 'mRS_premorbid_4', 
    'mRS_premorbid_3', 'Age_65', 'mRS_discharge_4', 'mRS_discharge_3', 
    'Inhospital_mortality', 'Seizure_terminated', 'mRS_3mo_4', 'mRS_3mo_3'
]

# 범주가 여러개인 변수 설정
multi_var = ['Etiology', 'Sz_type']

# 연속형 변수 설정 - 쉼표 추가
continuous_var = ['ASM_number', 'Age', 'STESS', 
                  'mRS_premorbid', 'Max_Mg_conc', 'Day_until_Mg', 
                  'mRS_discharge', 'HOD', 'RSE_duration', 'mRS_3mo']

# 분석할 variable의 순서 나열
var_order = ['STESS_5', 'STESS_3', 'SRSE', 'Sex', 'ImmunoTx', 'ASM_number',
             'Age', 'NORSE', 'STESS', 'mRS_premorbid', 'Mg_protocol',
             'Day_until_Mg', 'Max_Mg_conc', 'Age_65', 'mRS_premorbid_4',
             'mRS_premorbid_3', 'Etiology', 'Sz_type', 'RSE_duration',
             'HOD', 'mRS_3mo', 'mRS_3mo_4', 'mRS_3mo_3']

# 저장할 df 생성
columns = ['variable', 'responder', 'non-responder']
df_results = pd.DataFrame(columns=columns)

# total subject
total_subject = feature.shape[0]
total_responder = feature[target_response].sum().astype('int')
total_nonresponder = total_subject - total_responder.astype('int')

new_row = {'variable': 'total_subject',
           'responder': f"{total_responder} ({total_responder*100 / total_subject:.1f}%)",
           'non-responder': f"{total_nonresponder} ({total_nonresponder*100 / total_subject:.1f}%)"
           }
df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)

# 각 변수에 대한 함수 수행
for var in var_order:
    # 데이터프레임에 변수가 존재하는지 확인
    if var not in feature.columns:
        print(f"경고: '{var}' 변수가 데이터프레임에 존재하지 않습니다. 이 변수는 건너뜁니다.")
        continue
        
    if var in bi_var:
        try:
            cross = pd.crosstab(feature[var], feature[target_response])
            value_responder = cross.iloc[1, 1]
            value_nonresponder = cross.iloc[1, 0]
            
            new_row = {'variable': var,
               'responder': f"{value_responder} ({value_responder*100 / total_responder:.1f}%)",
               'non-responder': f"{value_nonresponder} ({value_nonresponder*100 / total_nonresponder:.1f}%)"
               }
            df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
        except Exception as e:
            print(f"'{var}' 변수 처리 중 오류 발생: {str(e)}")
    
    elif var in multi_var:
        new_row = {'variable': var}
        df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)    
        
        if var == 'Etiology':
            try:
                cross = pd.crosstab(feature[var], feature[target_response])
                # Etiology 카테고리 분석 (각 카테고리에 대해 인덱스 범위 확인)
                categories = cross.index.tolist()
                for i, category in enumerate(categories):
                    value_responder = cross.iloc[i, 1]
                    value_nonresponder = cross.iloc[i, 0]
                    new_row = {'variable': category,
                        'responder': f"{value_responder} ({value_responder*100 / total_responder:.1f}%)",
                        'non-responder': f"{value_nonresponder} ({value_nonresponder*100 / total_nonresponder:.1f}%)"
                        }            
                    df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
            except Exception as e:
                print(f"'Etiology' 변수 처리 중 오류 발생: {str(e)}")
        
        if var == 'Sz_type':
            try:
                cross = pd.crosstab(feature[var], feature[target_response])
                # Sz_type 카테고리 분석 (각 카테고리에 대해 인덱스 범위 확인)
                categories = cross.index.tolist()
                for i, category in enumerate(categories):
                    value_responder = cross.iloc[i, 1]
                    value_nonresponder = cross.iloc[i, 0]
                    new_row = {'variable': category,
                        'responder': f"{value_responder} ({value_responder*100 / total_responder:.1f}%)",
                        'non-responder': f"{value_nonresponder} ({value_nonresponder*100 / total_nonresponder:.1f}%)"
                        }            
                    df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
            except Exception as e:
                print(f"'Sz_type' 변수 처리 중 오류 발생: {str(e)}")

    elif var in continuous_var:
        try:
            # 연속형 변수 처리 - var를 col 대신 사용
            # 임시 변수 생성으로 continuous_var 목록 수정 방지
            feature_resp = feature.loc[feature[target_response]==1]
            feature_nonresp = feature.loc[feature[target_response]==0]
            
            new_row = {'variable': var,
                'responder': f"{feature_resp[var].median():.1f} ({feature_resp[var].quantile(0.25):.1f} - {feature_resp[var].quantile(0.75):.1f})",
                'non-responder': f"{feature_nonresp[var].median():.1f} ({feature_nonresp[var].quantile(0.25):.1f} - {feature_nonresp[var].quantile(0.75):.1f})"
                }         
            df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
        except Exception as e:
            print(f"'{var}' 연속형 변수 처리 중 오류 발생: {str(e)}")

# 결과를 엑셀 파일로 저장
try:
    df_results.to_excel(output_file_path, index=False)
    print(f"결과가 '{output_file_path}'에 성공적으로 저장되었습니다.")
except Exception as e:
    print(f"결과 저장 중 오류 발생: {str(e)}")

## 통계값 구하는 코드

이 코드는 엑셀로 내보내는데 실패... 직접 복사 붙여넣기 필요

In [ ]:
# 통계값 구하기. 이것은 직접 복사 붙여넣기 해야 합니다. 

import pandas as pd
import scipy.stats as stats

# clinical feature의 file_path 직접 입력
file_path = r"E:\주성\documents\2025\Mg comma\차트리뷰\clinical features10.xlsx"
feature = pd.read_excel(file_path, sheet_name = 'clinical_data')

# target_response 직접 입력
target_response = 'response_10_and'
print(target_response)

# 파생변수 생성. 파생변수 추가 시 여기에다가 추가
feature['STESS_5'] = feature['STESS'] >= 5
feature['STESS_3'] = feature['STESS'] >= 3
feature['mRS_premorbid_4'] = feature['mRS_premorbid'] >= 4
feature['mRS_premorbid_3'] = feature['mRS_premorbid'] >= 3
feature['Age_65'] = feature['Age'] >= 65
feature['mRS_discharge_4'] = feature['mRS_discharge'] >= 4
feature['mRS_discharge_3'] = feature['mRS_discharge'] >= 3
feature['mRS_3mo_4'] = feature['mRS_3mo'] >= 4
feature['mRS_3mo_3'] = feature['mRS_3mo'] >= 3
feature['Seizure_terminated'] = ~feature['Seizure_not_terminated']

############################################################################
############################################################################

results = {}
# 범주형
feature_cat = feature[['SRSE', 'CIVAD', 'Sex', 'ImmunoTx',
    'NORSE', 'Mg_protocol', 'STESS_5', 'STESS_3', 'mRS_premorbid_4', 
    'mRS_premorbid_3', 'Age_65', 'mRS_discharge_4', 'mRS_discharge_3', 
    'Inhospital_mortality', 'Seizure_terminated', 'mRS_3mo_4', 'mRS_3mo_3'
]]

for col in feature_cat.columns:
    cross = pd.crosstab(feature_cat[col], feature[target_response])
    s, p = stats.fisher_exact(cross)
    results[col] = {'p-value': round(p, 3)}

# 다변수 범주형
feature_mulvar = feature[['Etiology', 'Sz_type']]

for col in feature_mulvar.columns:
    table = pd.crosstab(feature_mulvar[col], feature[target_response])
    s, p, d, e = stats.chi2_contingency(table)
    results[col] = {'p-value': round(p, 3)}

# 연속형 
feature_num = feature[['ASM_number', 'Age', 'STESS',  'mRS_premorbid', 'Max_Mg_conc', target_response, 'mRS_discharge', 'HOD', 'RSE_duration']]

for col in feature_num.columns:
    x = feature_num.loc[feature_num[target_response]== True][col]
    y = feature_num.loc[feature_num[target_response]== False][col]
    statistics, p_value = stats.mannwhitneyu(x, y)
    results[col] = {'p-value': round(p_value, 3)}

feature2 = feature.copy().dropna(subset='mRS_3mo')
feature2_1 = feature2.loc[feature2[target_response]==1]
feature2_0 = feature2.loc[feature2[target_response]==0]

s, p = stats.mannwhitneyu(feature2_1['mRS_3mo'], feature2_0['mRS_3mo'])
results['mRS_3mo'] = round(p, 3)

cross_4 = pd.crosstab(feature2['mRS_3mo_4'], feature2[target_response])
s, p = stats.fisher_exact(cross_4)
results['mRS_3mo_44'] = round(p, 3)

cross_3 = pd.crosstab(feature2['mRS_3mo_3'], feature2[target_response])
s, p = stats.fisher_exact(cross_3)
results['mRS_3mo_33'] = round(p, 3)

result_df = pd.DataFrame(results).T

## Partial response 포함하는 코드... (사용할지는 미정)

In [ ]:
import pandas as pd

# clinical feature
feature = pd.read_excel(r"E:\주성\documents\2025\Mg comma\차트리뷰\clinical features12.xlsx", sheet_name = 'clinical_data')

# target_response
target_response = "response_10_tri"
print(target_response)

# 파생변수
feature['STESS_5'] = feature['STESS'] >= 5
feature['STESS_3'] = feature['STESS'] >= 3
feature['mRS_premorbid_4'] = feature['mRS_premorbid'] >= 4
feature['mRS_premorbid_3'] = feature['mRS_premorbid'] >= 3
feature['Age_65'] = feature['Age'] >=65
feature['mRS_discharge_4'] = feature['mRS_discharge'] >= 4
feature['mRS_discharge_3'] = feature['mRS_discharge'] >= 3
feature['mRS_3mo_4'] = feature['mRS_3mo'] >= 4
feature['mRS_3mo_3'] = feature['mRS_3mo'] >= 3

# Total subject
for i in ['TRUE', 'PART', 'FALSE']:
    i_subject = feature.loc[feature[target_response] == i].shape[0]
    print(f"{i} : {i_subject}")

# 범주형
feature_cat = feature[['SRSE', 'CIVAD', 'Sex', 'ImmunoTx',
    'NORSE', 'Mg_protocol', 'STESS_5', 'STESS_3', 'mRS_premorbid_4', 'mRS_premorbid_3', 'Age_65', 'Etiology', 
    'Sz_type', 'Seizure_not_terminated', 'mRS_discharge_4', 'mRS_discharge_3', 'Inhospital_mortality'
]]

for col in feature_cat.columns:
    cross = pd.crosstab(feature_cat[col], feature[target_response])
    print(cross)

# 연속형
feature_num = feature[['ASM_number', 'Age', 'STESS',  'mRS_premorbid', 'Max_Mg_conc', target_response, 'mRS_discharge', 'HOD', 'RSE_duration', 'Day_until_Mg']]

feature_num_TRUE = feature_num.loc[feature_num[target_response]=='TRUE']
feature_num_PART = feature_num.loc[feature_num[target_response]=='PART']
feature_num_FALSE = feature_num.loc[feature_num[target_response]=='FALSE']

feature_cols = [col for col in feature_num.columns if col != target_response]

for col in feature_cols:
    print(col)
    print('response')
    print(f"{feature_num_TRUE[col].median()} ({feature_num_TRUE[col].quantile(0.25)} - {feature_num_TRUE[col].quantile(0.75)})")
    print('partial response')
    print(f"{feature_num_PART[col].median()} ({feature_num_PART[col].quantile(0.25)} - {feature_num_PART[col].quantile(0.75)})")
    print('non-response')
    print(f"{feature_num_FALSE[col].median()} ({feature_num_FALSE[col].quantile(0.25)} - {feature_num_FALSE[col].quantile(0.75)})")    

# 3mo는 따로 계산
feature2 = feature.copy().dropna(subset='mRS_3mo')
feature2_1 = feature2.loc[feature2[target_response]=='TRUE']
feature2_0 = feature2.loc[feature2[target_response]=='FALSE']
feature2_p = feature2.loc[feature2[target_response]=='PART']
print('mRS_3mo')
print('response')
print(f"{feature2_1['mRS_3mo'].median()} ({feature2_1['mRS_3mo'].quantile(0.25)} - {feature2_1['mRS_3mo'].quantile(0.75)})")
print('partial response')
print(f"{feature2_p['mRS_3mo'].median()} ({feature2_p['mRS_3mo'].quantile(0.25)} - {feature2_p['mRS_3mo'].quantile(0.75)})")
print('non-response')
print(f"{feature2_0['mRS_3mo'].median()} ({feature2_0['mRS_3mo'].quantile(0.25)} - {feature2_0['mRS_3mo'].quantile(0.75)})")
cross_3mo_3 = pd.crosstab(feature2['mRS_3mo_3'], feature2[target_response])
print('mRS_3mo_3')
print(cross_3mo_3)
cross_3mo_4 = pd.crosstab(feature2['mRS_3mo_4'], feature2[target_response])
print('mRS_3mo_4')
print(cross_3mo_4)

# Table 2

In [ ]:
import pandas as pd

# file_path 입력
input_file_path = r"C:\Users\Brain_Science\Documents\박주성\Mg comma\차트리뷰\clinical features11.xlsx"
df = pd.read_excel(input_file_path, sheet_name='pharmacoEEG')

# 저장할 컬럼명
columns = ['band', 'post_median', 'day1_median', 'post_ratio', 'day1_ratio']

# 내보내는 경로
output_file_path = r"C:\Users\Brain_Science\Documents\박주성\Mg comma\차트리뷰\eeg_results.xlsx"

###############################################################################

# 각 밴드별 데이터프레임 만들기
total = df.copy()
delta = df.loc[df['band']=='delta']
theta = df.loc[df['band']=='theta']
alpha = df.loc[df['band']=='alpha']
beta = df.loc[df['band']=='beta']
gamma = df.loc[df['band']=='gamma']

# 저장할 데이터프레임
df_results = pd.DataFrame(columns=columns)

for band in ['total', 'delta', 'theta', 'alpha', 'beta', 'gamma']:
    if band == 'total':
        new_row = {'band': band, 
               'post_median': f"{df['post_change'].median()*100:.1f}% ({df['post_change'].quantile(0.25)*100:.1f} - {df['post_change'].quantile(0.75)*100:.1f})", 
               'day1_median': f"{df['day1_change'].median()*100:.1f}% ({df['day1_change'].quantile(0.25)*100:.1f} - {df['day1_change'].quantile(0.75)*100:.1f})",
               'post_ratio': f"{(df['post_change']<0).sum()} ({round((df['post_change']<0).sum()*100/100, 1)}%)",
               'day1_ratio': f"{(df['day1_change']<0).sum()} ({round((df['day1_change']<0).sum()*100/90, 1)}%)"
               }
        df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
    
    else:
        band_df = df.loc[df['band']==band]        
        new_row = {'band': band, 
            'post_median': f"{band_df['post_change'].median()*100:.1f}% ({band_df['post_change'].quantile(0.25)*100:.1f} - {band_df['post_change'].quantile(0.75)*100:.1f})", 
            'day1_median': f"{band_df['day1_change'].median()*100:.1f}% ({band_df['day1_change'].quantile(0.25)*100:.1f} - {band_df['day1_change'].quantile(0.75)*100:.1f})",
            'post_ratio': f"{(band_df['post_change']<0).sum()} ({round((band_df['post_change']<0).sum()*100/20, 1)}%)",
            'day1_ratio': f"{(band_df['day1_change']<0).sum()} ({round((band_df['day1_change']<0).sum()*100/90, 1)}%)"
            }
        df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)

df_results.to_excel(output_file_path, index=False)

print(df_results)        

In [ ]:
df = pd.read_excel(r"D:\주성\documents\2025\Mg comma\clinical features6.xlsx", sheet_name='Sheet1')
df = df.iloc[:-2, :]

df1 = df[['Mg type', 'mRS_Discharge', 'mRS_3mo_text', 'Inhospital_Mortality', 'SE_Duration', 'Hospital_days']]
df1['mRS_Discharge_4'] = df1['mRS_Discharge'] >=4
df1['mRS_Discharge_5'] = df1['mRS_Discharge'] >=5

for col in ['mRS_Discharge', 'SE_Duration', 'Hospital_days']:
    x = df1.loc[df1['Mg type']==1][col]
    y = df1.loc[df1['Mg type']==2][col]
    s, p = stats.mannwhitneyu(x, y)
    print(col, p, x.median(), x.quantile(0.25), x.quantile(0.75), y.median(), y.quantile(0.25), y.quantile(0.75))

for col in ['mRS_Discharge_4', 'Inhospital_Mortality', 'mRS_Discharge_5']:
    cross = pd.crosstab(df1['Mg type'], df1[col])
    s, p = stats.fisher_exact(cross)
    print(col, cross, p)

# Table 4

## 기술적 통계값

In [ ]:
import pandas as pd
import numpy as np

# 변수 설정 부분

# clinical feature 파일의 경로 설정
input_file_path = r"D:\주성\documents\2025\Mg comma\차트리뷰\clinical features12.xlsx"

# 결과를 저장할 엑셀 파일의 경로 설정
output_file_path = r"C:\Users\Brain_Science\Documents\박주성\Mg comma\표그림\new.xlsx"

# 분석할 response 컬럼 설정
target_response = 'Mg_protocol'

# clinical feature 불러오기
feature = pd.read_excel(input_file_path, sheet_name='clinical_data')
# 결측치 제거 - 수정된 부분: 결과를 feature에 다시 할당
feature = feature.dropna(subset=[target_response])

# 파생변수 생성. 파생변수 추가 시 여기에다가 추가
feature['STESS_5'] = feature['STESS'] >= 5
feature['STESS_3'] = feature['STESS'] >= 3
feature['mRS_premorbid_4'] = feature['mRS_premorbid'] >= 4
feature['mRS_premorbid_3'] = feature['mRS_premorbid'] >= 3
feature['Age_65'] = feature['Age'] >= 65
feature['mRS_discharge_4'] = feature['mRS_discharge'] >= 4
feature['mRS_discharge_3'] = feature['mRS_discharge'] >= 3
feature['mRS_3mo_4'] = feature['mRS_3mo'] >= 4
feature['mRS_3mo_3'] = feature['mRS_3mo'] >= 3
feature['Seizure_terminated'] = ~feature['Seizure_not_terminated']

# 범주가 2개인 변수 설정
bi_var = ['SRSE', 'CIVAD', 'Sex', 'ImmunoTx',
    'NORSE', 'STESS_5', 'STESS_3', 'mRS_premorbid_4', 
    'mRS_premorbid_3', 'Age_65', 'mRS_discharge_4', 'mRS_discharge_3', 
    'Inhospital_mortality', 'Seizure_terminated', 'mRS_3mo_4', 'mRS_3mo_3'
]

# 범주가 여러개인 변수 설정
multi_var = ['Etiology', 'Sz_type']

# 연속형 변수 설정 - 쉼표 추가
continuous_var = ['ASM_number', 'Age', 'STESS', 
                  'mRS_premorbid', 'Max_Mg_conc', 'Day_until_Mg', 
                  'mRS_discharge', 'HOD', 'RSE_duration', 'mRS_3mo']

# 분석할 variable의 순서 나열
var_order = ['STESS_5', 'STESS_3', 'SRSE', 'Sex', 'ImmunoTx', 'ASM_number',
             'Age', 'NORSE', 'STESS', 'mRS_premorbid', 'Mg_protocol',
             'Day_until_Mg', 'Max_Mg_conc', 'Age_65', 'mRS_premorbid_4',
             'mRS_premorbid_3', 'Etiology', 'Sz_type', 'RSE_duration',
             'HOD', 'mRS_3mo', 'mRS_3mo_4', 'mRS_3mo_3']

##########################################################################

# 저장할 df 생성
columns = ['variable', 'Protocol A', 'Protocol B']
df_results = pd.DataFrame(columns=columns)

# total subject
total_subject = feature.shape[0]
total_responder = feature[target_response].sum().astype('int')
total_nonresponder = total_subject - total_responder.astype('int')

new_row = {'variable': 'total_subject',
           'responder': f"{total_responder} ({total_responder*100 / total_subject:.1f}%)",
           'non-responder': f"{total_nonresponder} ({total_nonresponder*100 / total_subject:.1f}%)"
           }
df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)

# 각 변수에 대한 함수 수행
for var in var_order:
    # 데이터프레임에 변수가 존재하는지 확인
    if var not in feature.columns:
        print(f"경고: '{var}' 변수가 데이터프레임에 존재하지 않습니다. 이 변수는 건너뜁니다.")
        continue
        
    if var in bi_var:
        try:
            cross = pd.crosstab(feature[var], feature[target_response])
            value_responder = cross.iloc[1, 1]
            value_nonresponder = cross.iloc[1, 0]
            
            new_row = {'variable': var,
               'responder': f"{value_responder} ({value_responder*100 / total_responder:.1f}%)",
               'non-responder': f"{value_nonresponder} ({value_nonresponder*100 / total_nonresponder:.1f}%)"
               }
            df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
        except Exception as e:
            print(f"'{var}' 변수 처리 중 오류 발생: {str(e)}")
    
    elif var in multi_var:
        new_row = {'variable': var}
        df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)    
        
        if var == 'Etiology':
            try:
                cross = pd.crosstab(feature[var], feature[target_response])
                # Etiology 카테고리 분석 (각 카테고리에 대해 인덱스 범위 확인)
                categories = cross.index.tolist()
                for i, category in enumerate(categories):
                    value_responder = cross.iloc[i, 1]
                    value_nonresponder = cross.iloc[i, 0]
                    new_row = {'variable': category,
                        'responder': f"{value_responder} ({value_responder*100 / total_responder:.1f}%)",
                        'non-responder': f"{value_nonresponder} ({value_nonresponder*100 / total_nonresponder:.1f}%)"
                        }            
                    df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
            except Exception as e:
                print(f"'Etiology' 변수 처리 중 오류 발생: {str(e)}")
        
        if var == 'Sz_type':
            try:
                cross = pd.crosstab(feature[var], feature[target_response])
                # Sz_type 카테고리 분석 (각 카테고리에 대해 인덱스 범위 확인)
                categories = cross.index.tolist()
                for i, category in enumerate(categories):
                    value_responder = cross.iloc[i, 1]
                    value_nonresponder = cross.iloc[i, 0]
                    new_row = {'variable': category,
                        'responder': f"{value_responder} ({value_responder*100 / total_responder:.1f}%)",
                        'non-responder': f"{value_nonresponder} ({value_nonresponder*100 / total_nonresponder:.1f}%)"
                        }            
                    df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
            except Exception as e:
                print(f"'Sz_type' 변수 처리 중 오류 발생: {str(e)}")

    elif var in continuous_var:
        try:
            # 연속형 변수 처리 - var를 col 대신 사용
            # 임시 변수 생성으로 continuous_var 목록 수정 방지
            feature_resp = feature.loc[feature[target_response]==1]
            feature_nonresp = feature.loc[feature[target_response]==0]
            
            new_row = {'variable': var,
                'responder': f"{feature_resp[var].median():.1f} ({feature_resp[var].quantile(0.25):.1f} - {feature_resp[var].quantile(0.75):.1f})",
                'non-responder': f"{feature_nonresp[var].median():.1f} ({feature_nonresp[var].quantile(0.25):.1f} - {feature_nonresp[var].quantile(0.75):.1f})"
                }         
            df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
        except Exception as e:
            print(f"'{var}' 연속형 변수 처리 중 오류 발생: {str(e)}")

# 결과를 엑셀 파일로 저장
try:
    df_results.to_excel(output_file_path, index=False)
    print(f"결과가 '{output_file_path}'에 성공적으로 저장되었습니다.")
except Exception as e:
    print(f"결과 저장 중 오류 발생: {str(e)}")

## 통계값 구하는 코드. 복붙해야함

In [ ]:
# 통계값 구하기. 이것은 직접 복사 붙여넣기 해야 합니다. 

import pandas as pd
import scipy.stats as stats

# clinical feature의 file_path 직접 입력
file_path = r"C:\Users\Brain_Science\Documents\박주성\Mg comma\차트리뷰\clinical features11.xlsx"
feature = pd.read_excel(file_path, sheet_name = 'clinical_data')

# target_response 직접 입력
target_response = 'Mg_protocol'
print(target_response)

# 파생변수 생성. 파생변수 추가 시 여기에다가 추가
feature['STESS_5'] = feature['STESS'] >= 5
feature['STESS_3'] = feature['STESS'] >= 3
feature['mRS_premorbid_4'] = feature['mRS_premorbid'] >= 4
feature['mRS_premorbid_3'] = feature['mRS_premorbid'] >= 3
feature['Age_65'] = feature['Age'] >= 65
feature['mRS_discharge_4'] = feature['mRS_discharge'] >= 4
feature['mRS_discharge_3'] = feature['mRS_discharge'] >= 3
feature['mRS_3mo_4'] = feature['mRS_3mo'] >= 4
feature['mRS_3mo_3'] = feature['mRS_3mo'] >= 3
feature['Seizure_terminated'] = ~feature['Seizure_not_terminated']

############################################################################
############################################################################

results = {}
# 범주형
feature_cat = feature[['SRSE', 'CIVAD', 'Sex', 'ImmunoTx',
    'NORSE', 'STESS_5', 'STESS_3', 'mRS_premorbid_4', 
    'mRS_premorbid_3', 'Age_65', 'mRS_discharge_4', 'mRS_discharge_3', 
    'Inhospital_mortality', 'Seizure_terminated', 'mRS_3mo_4', 'mRS_3mo_3'
]]

for col in feature_cat.columns:
    cross = pd.crosstab(feature_cat[col], feature[target_response])
    s, p = stats.fisher_exact(cross)
    results[col] = {'p-value': round(p, 3)}

# 다변수 범주형
feature_mulvar = feature[['Etiology', 'Sz_type']]

for col in feature_mulvar.columns:
    table = pd.crosstab(feature_mulvar[col], feature[target_response])
    s, p, d, e = stats.chi2_contingency(table)
    results[col] = {'p-value': round(p, 3)}

# 연속형 
feature_num = feature[['ASM_number', 'Age', 'STESS',  'mRS_premorbid', 'Max_Mg_conc', target_response, 'mRS_discharge', 'HOD', 'RSE_duration']]

for col in feature_num.columns:
    x = feature_num.loc[feature_num[target_response]== True][col]
    y = feature_num.loc[feature_num[target_response]== False][col]
    statistics, p_value = stats.mannwhitneyu(x, y)
    results[col] = {'p-value': round(p_value, 3)}

feature2 = feature.copy().dropna(subset='mRS_3mo')
feature2_1 = feature2.loc[feature2[target_response]==1]
feature2_0 = feature2.loc[feature2[target_response]==0]

s, p = stats.mannwhitneyu(feature2_1['mRS_3mo'], feature2_0['mRS_3mo'])
results['mRS_3mo'] = round(p, 3)

cross_4 = pd.crosstab(feature2['mRS_3mo_4'], feature2[target_response])
s, p = stats.fisher_exact(cross_4)
results['mRS_3mo_44'] = round(p, 3)

cross_3 = pd.crosstab(feature2['mRS_3mo_3'], feature2[target_response])
s, p = stats.fisher_exact(cross_3)
results['mRS_3mo_33'] = round(p, 3)

result_df = pd.DataFrame(results).T

result_df